In [3]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import cv2
import MPS

In [6]:
x = MPS.mps.init_rand(d = 2, chi = 10, length = 10)
# :param d: 物理指标维数
# :param chi: 辅助指标截断维数
# :param length: 张量个数

for n in range(0,x.length):
    print(x.tensors[n].shape)
    # tensors is a list

(2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2)


In [10]:
tensors = list()
tensors += [np.random.randn(3, 5)]
tensors += [np.random.randn(5, 10, 9)]
tensors += [np.random.randn(9, 2, 7)]
tensors += [np.random.randn(7, 10, 3)]
tensors += [np.random.randn(3, 6)]
y = MPS.mps.init_tensors(tensors)
for n in range(0,y.length):
    print(y.tensors[n].shape)

(3, 5)
(5, 10, 9)
(9, 2, 7)
(7, 10, 3)
(3, 6)


In [8]:
X = x.mps2tensor()
x.center_orth(1, cut_dim = -1)
for n in range(0,x.length):
    print(x.tensors[n].shape)

(2, 2)
(2, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 8)
(8, 2, 4)
(4, 2, 2)
(2, 2)


In [9]:
X1 = x.mps2tensor()
X = X/np.linalg.norm(X)
X1 = X1/np.linalg.norm(X1)
err = np.linalg.norm(X-X1)
print(err)

3.143713616191264e-15


In [11]:
Y = y.mps2tensor()
y.center_orth(1, cut_dim = 5)
for n in range(0,y.length):
    print(y.tensors[n].shape)

(3, 3)
(3, 10, 5)
(5, 2, 5)
(5, 10, 3)
(3, 6)


In [12]:
Y1 = y.mps2tensor()
Y = Y/np.linalg.norm(Y)
Y1 = Y1/np.linalg.norm(Y1)
err = np.linalg.norm(Y-Y1)
print(err)

0.2818636648491108
